In [1]:
# Importações
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_recall_curve, roc_curve, f1_score, 
    precision_score, recall_score
)
import warnings
warnings.filterwarnings('ignore')

In [3]:
os.chdir('src')
from data.load_data import carregar_dados

print(os.getcwd())
df = carregar_dados("../data/raw/v1/bs140513_032310.csv")

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'src'

In [23]:
# Distribuição da variável target
fraud_counts = df['fraud'].value_counts().sort_index()
fraud_pct = df['fraud'].value_counts(normalize=True).sort_index() * 100

for value in sorted(df['fraud'].unique()):
    count = fraud_counts[value]
    pct = fraud_pct[value]
    label = 'Normal' if value == 0 else 'Fraude'

In [24]:
# Criando uma cópia do dataframe para feature engineering
df_features = df.copy()

In [25]:
# ============================================================================
# 1. FEATURES BASEADAS NO CLIENTE
# ============================================================================

# Frequência de transações por cliente no mesmo step
freq_step = (
    df_features.groupby(['step', 'customer'])
    .size()
    .reset_index(name='qtd_transacoes')
)
df_features = df_features.merge(freq_step, on=['step', 'customer'])
df_features['alert_freq'] = (df_features['qtd_transacoes'] > 3).astype(int)

# Perfil de valor por cliente (média e desvio padrão)
stats_cliente = (
    df_features.groupby('customer')['amount']
    .agg(['mean', 'std'])
    .reset_index()
)
stats_cliente.columns = ['customer', 'amount_mean_cliente', 'amount_std_cliente']
df_features = df_features.merge(stats_cliente, on='customer')
df_features['amount_std_cliente'].fillna(0, inplace=True)
df_features['alert_valor'] = (
    df_features['amount'] > (df_features['amount_mean_cliente'] + 3 * df_features['amount_std_cliente'])
).astype(int)

# Valor relativo à média do cliente
df_features['valor_relativo_cliente'] = df_features['amount'] / (df_features['amount_mean_cliente'] + 1e-6)

# Total de transações por cliente
df_features['total_tx_cliente'] = (
    df_features.groupby('customer')['amount']
    .transform('count')
)

# Volume total gasto pelo cliente
df_features['volume_total_cliente'] = (
    df_features.groupby('customer')['amount']
    .transform('sum')
)

# Diversidade de categorias por cliente
df_features['num_categorias_cliente'] = (
    df_features.groupby('customer')['category']
    .transform('nunique')
)

# Diversidade de merchants por cliente
df_features['num_merchants_cliente'] = (
    df_features.groupby('customer')['merchant']
    .transform('nunique')
)

In [26]:
# ============================================================================
# 2. FEATURES TEMPORAIS
# ============================================================================

# Ordenando por cliente e step
df_features = df_features.sort_values(['customer', 'step']).reset_index(drop=True)

# Transações nos últimos 5 steps
df_features['tx_ultimos_5_steps'] = (
    df_features.groupby('customer')['step']
    .transform(lambda x: x.rolling(5, min_periods=1).count())
)

# Tempo desde a última transação
df_features['step_diff'] = (
    df_features.groupby('customer')['step']
    .diff()
    .fillna(0)
)

# Média de valor dos últimos 5 steps
df_features['amount_media_5steps'] = (
    df_features.groupby('customer')['amount']
    .transform(lambda x: x.rolling(5, min_periods=1).mean())
)

# Desvio do valor atual em relação aos últimos 5 steps
df_features['amount_desvio_5steps'] = (
    df_features['amount'] - df_features['amount_media_5steps']
)

In [27]:
# ============================================================================
# 4. FEATURES DE RELACIONAMENTO CLIENTE-MERCHANT
# ============================================================================

# Frequência do par cliente-merchant
df_features['tx_cliente_merchant'] = (
    df_features.groupby(['customer', 'merchant'])['amount']
    .transform('count')
)

# É a primeira transação deste cliente com este merchant?
df_features['primeira_tx_merchant'] = (
    df_features['tx_cliente_merchant'] == 1
).astype(int)

# Proporção de transações do cliente neste merchant
df_features['prop_tx_merchant'] = (
    df_features['tx_cliente_merchant'] / df_features['total_tx_cliente']
)

In [28]:
# ============================================================================
# 5. FEATURES DE LOCALIZAÇÃO
# ============================================================================


# Mesma localização?
df_features['mesma_localizacao'] = (
    df_features['zipcodeOri'] == df_features['zipMerchant']
).astype(int)

# Número de diferentes localizações do cliente
df_features['num_zipcodes_cliente'] = (
    df_features.groupby('customer')['zipcodeOri']
    .transform('nunique')
)

In [30]:
# Encoding de variáveis categóricas
le_gender = LabelEncoder()
le_category = LabelEncoder()

df_features['gender_encoded'] = le_gender.fit_transform(df_features['gender'])
df_features['category_encoded'] = le_category.fit_transform(df_features['category'])

In [31]:
# Selecionando features para o modelo
features_to_use = [
    # Features originais
    'step', 'age', 'gender_encoded', 'category_encoded', 'amount',
    
    # Features engineered - Cliente
    'qtd_transacoes', 'alert_freq', 'alert_valor', 'valor_relativo_cliente',
    'total_tx_cliente', 'volume_total_cliente', 'num_categorias_cliente',
    'num_merchants_cliente', 'amount_mean_cliente', 'amount_std_cliente',
    
    # Features temporais
    'tx_ultimos_5_steps', 'step_diff', 'amount_media_5steps', 'amount_desvio_5steps',
    
    # Features merchant
    #'tx_por_merchant_train', 'fraude_merchant_train', 'amount_mean_merchant'
     'amount_std_merchant',
    
    # Features relacionamento
    'tx_cliente_merchant', 'primeira_tx_merchant', 'prop_tx_merchant',
    
    # Features localização
    'mesma_localizacao', 'num_zipcodes_cliente',
    
    # Features categoria
    #'fraude_categoria'
     'amount_mean_categoria', 'amount_desvio_categoria',
    
    # Scores
    'qtd_alertas', 'score_regra'
]

# Verificar se todas as features existem
missing_features = [f for f in features_to_use if f not in df_features.columns]
if missing_features:
    features_to_use = [f for f in features_to_use if f in df_features.columns]

X = df_features[features_to_use].copy()
y = df_features['fraud'].copy()

# Garantir que X contém apenas valores numéricos
X = X.apply(pd.to_numeric, errors='coerce')

# Substituir inf e -inf por NaN
X = X.replace([np.inf, -np.inf], np.nan)

# Preencher NaN com 0
X = X.fillna(0)

In [32]:
# Split treino/teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [33]:
# Normalização das features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
# Função para avaliar modelos
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Treina e avalia um modelo de classificação"""
    
    # Treinamento
    model.fit(X_train, y_train)
    
    # Predições
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    return {
        'model': model,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'cm': cm
    }

In [35]:
# 1. Logistic Regression
lr_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'
)

lr_results = evaluate_model(
    lr_model, X_train_scaled, X_test_scaled, y_train, y_test,
    "Logistic Regression"
)

In [ ]:
# 2. Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_results = evaluate_model(
    rf_model, X_train, X_test, y_train, y_test,
    "Random Forest"
)

In [19]:
# 3. Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)

gb_results = evaluate_model(
    gb_model, X_train, X_test, y_train, y_test,
    "Gradient Boosting"
)


Treinando: Gradient Boosting

Métricas no conjunto de teste:
  Precision: 0.9180
  Recall:    0.8007
  F1-Score:  0.8553
  ROC-AUC:   0.9982

Matriz de Confusão:
[[117386    103]
 [   287   1153]]

Relatório de Classificação:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00    117489
      Fraude       0.92      0.80      0.86      1440

    accuracy                           1.00    118929
   macro avg       0.96      0.90      0.93    118929
weighted avg       1.00      1.00      1.00    118929



In [20]:
# Comparação de métricas
comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    'Precision': [lr_results['precision'], rf_results['precision'], gb_results['precision']],
    'Recall': [lr_results['recall'], rf_results['recall'], gb_results['recall']],
    'F1-Score': [lr_results['f1'], rf_results['f1'], gb_results['f1']],
    'ROC-AUC': [lr_results['roc_auc'], rf_results['roc_auc'], gb_results['roc_auc']]
})

print("\n" + "="*70)
print("COMPARAÇÃO DE MODELOS")
print("="*70)
print(comparison.to_string(index=False))
print("="*70)


COMPARAÇÃO DE MODELOS
              Model  Precision   Recall  F1-Score  ROC-AUC
Logistic Regression   0.246814 0.981944  0.394476 0.995627
      Random Forest   0.730546 0.873611  0.795699 0.998023
  Gradient Boosting   0.917994 0.800694  0.855341 0.998225
